In [7]:
import pandas
from bokeh.io import output_notebook

In [8]:
# render inline
output_notebook()

Loading BokehJS ...

In [9]:
# Column names from our CSV
colNames = ['S1', 'C1', 'S2', 'C2', 'S3', 'C3', 'S4', 'C4', 'S5', 'C5', 'CLASS']

# Read CSV file using pandas
data = pandas.read_csv('data.csv', names=colNames)

# Extract all data from the CLASS column
hands = data.CLASS.tolist()

# Remove the first element 
hands.pop(0)

' CLASS'

In [10]:
# Count occurances of each class
classZero   = hands.count('0') # Nothing in hand
classOne    = hands.count('1') # One pair
classTwo    = hands.count('2') # Two pair
classThree  = hands.count('3') # Three of a kind
classFour   = hands.count('4') # Straight
classFive   = hands.count('5') # Flush
classSix    = hands.count('6') # Full House
classSeven  = hands.count('7') # Four of a kind
classEight  = hands.count('8') # Straight Flush
classNine   = hands.count('9') # Royal Flush

In [11]:
dataset = [classZero, classOne, classTwo, classThree, classFour, classFive, classSix, classSeven, classEight, classNine]

In [12]:
# Create a bar chart

from bokeh.charts import Bar, output_file, show

# prepare some data
data = {"y": [6, 7, 2, 4, 5], "z": [1, 5, 12, 4, 2]}

# create a new line chat with a title and axis labels
p = Bar(data, cat=['C1', 'C2', 'C3', 'D1', 'D2'], title="Bar example", xlabel='categories', ylabel='values', width=400, height=400)

# show the results
show(p)

AttributeError: unexpected attribute 'cat' to Chart, possible attributes are above, background_fill_alpha, background_fill_color, below, border_fill_alpha, border_fill_color, disabled, extra_x_ranges, extra_y_ranges, h_symmetry, hidpi, left, legend, lod_factor, lod_interval, lod_threshold, lod_timeout, logo, min_border, min_border_bottom, min_border_left, min_border_right, min_border_top, name, outline_line_alpha, outline_line_cap, outline_line_color, outline_line_dash, outline_line_dash_offset, outline_line_join, outline_line_width, plot_height, plot_width, renderers, responsive, right, tags, title, title_standoff, title_text_align, title_text_alpha, title_text_baseline, title_text_color, title_text_font, title_text_font_size, title_text_font_style, tool_events, toolbar_location, tools, v_symmetry, webgl, x_mapper_type, x_range, xgrid, xlabel, xscale, y_mapper_type, y_range, ygrid, ylabel or yscale